In [43]:
#Rings
R.<x>=QQ[]
R2.<x,y>=QQ[]
R3.<x,y,z>=QQ[]
R4.<x,y,z,w>=QQ[]

In [44]:
#Input:  Ideal I in n variables
#Output: Mon(I)=⟨ m∈ I | m is a monomial ⟩

I=ideal(R3,[x^3+y^3+z^3,x+y+z])
Mon(I)

[x*y*z]

In [45]:
#Input:   Ideal I in n≤3 variables, t ∈ N  
#Output:  Bin(I_{≤t})=⟨ b∈ I_{≤t} | b is a binomial ⟩

I1=ideal(R3,[(x-z)^2,13*x-y-(12)*z])
I2=ideal(R3,[z^2+z-1,5*y+4*z+7,x-3*z+1])
print(Bin(I1,20))
print(Bin(I2,20))                              

[x^13 - y*z^12]
[x^3*y + 5*z^3, z^4 - 1/5*x^2, x*y*z + 1]


In [46]:
#Input:   f∈ Q[x], t ∈ N
#Output:  One binomial in ⟨f⟩_{≤t} or 0 if there is none

f=R(cyclotomic_polynomial(8)*cyclotomic_polynomial(3)*cyclotomic_polynomial(5))
BinUnivariate(f,1000)

x^120 - 1

In [47]:
#Input:   f∈ Q[x], t ∈ N
#Output:  One trinomial in ⟨f⟩_{≤t} or 0 if there is none

print(Trinomial(R(x^4-x^3+x^2+2),1000))
print(Trinomial(R(x^4-2*x^2+4*x+6),1000))
print(Trinomial(R(x^5-x^3+x^2+x+1),1000))

x^24 + 63*x^12 - 64
x^9 - 76*x - 96
x^8 + 3*x^3 - 1


In [48]:
#Input:   Ideal I,  L a basis of a lattice ⊆ Z^k
#Output:  Ideal J = I ∩ Q[L]

I=ideal(R4,[5*y^2*z^2+15*y*z-1,15*y*z^3+40*z^2+7*x,20*y^4*z+65*y^3+7*w])
L=[[1,0,-2,0],[0,3,0,-1],[0,2,-1,-1]]
getJ(I,L)

[a - 3*c + 1, b + 4/5*c + 7/5, c^2 + c - 1]

In [49]:
#Input:  Ideal I in n variables
#Output: Mon(I)=⟨ m∈ I | m is a monomial ⟩
def Mon(I):
    I=I.groebner_basis()
    R=I.ring()
    a=R.gens()
    n=R.ngens()
    Rbig=PolynomialRing(QQ,[('t'+str(k))  for k in range(n)]+list(a),order='lex')   
    m=Rbig(1)
    for k in range(n):
        m*=Rbig('t'+str(k))
    GB_big=[m-1]
    for f in I:
        f=Rbig(f)
        for k in range(n):
            f=f.subs({Rbig(a[k]):Rbig(a[k])*Rbig('t'+str(k))})
        GB_big+=[f]
    MonI=[]
    GB_sol=ideal(Rbig,GB_big).groebner_basis()
    for g in GB_sol:
        if g.lm() < Rbig("t0") and g.lt()==g:
            MonI+=[g]
    return Ideal(R,MonI).groebner_basis()



#Input:   Ideal I in n≤3 variables, t ∈ N     
#Output:  Bin(I_{≤t})=⟨ b∈ I_{≤t} | b is a binomial ⟩
def Bin(I,t):
    R=I.ring()
    n=R.ngens()
    if(n==1):
           return Bin(I[0],t)
    a=I.ring().gens()
    I=I.groebner_basis()
    P=[]
    if(n==2):
        IIh=[]
        L=LaurentPolynomialRing(QQ,a)
        for f in I:
            IIh+=[(f.subs({a[1] : a[1]^-1})/a[0])  .numerator()]
        II=ideal(R,ideal(L,IIh).groebner_basis() ).groebner_basis()
        for i in range(0,t+1):
            for j in range(0,t+1):
                if(not i==j==0):
                    XP=R(a[0]^i*a[1]^j)
                    if (XP).reduce(I) in QQ:
                        P+=[XP - (XP).reduce(I) ]
                    if (XP).reduce(II) in QQ:
                        P+=[R(a[0]^i) - a[1]^j*(XP).reduce(II)]
    if(n==3):
        L=LaurentPolynomialRing(QQ,a)
        II=[]
        for k in range(3):
            IIh=[]
            for f in I:
                IIh+=[(f.subs({a[k] : a[k]^-1})/a[0] )  .numerator()] 
            II.append(ideal(R,ideal(L,IIh).groebner_basis() ).groebner_basis())
        for i in range(0,t+1):
            for j in range(0,t+1):
                for s in range(0,t+1):  
                    if(not i==j==s==0 ):
                        X=[R(a[0]^i),R(a[1]^j),R(a[2]^s)]; XP=R(X[0]*X[1]*X[2])
                        if (XP).reduce(I) in QQ:
                            P+=[ XP - XP.reduce(I) ]
                        for k in range(3):       
                            if (XP).reduce(II[k]) in QQ:
                                P+=[X[(k+1)%3]*X[(k+2)%3] - X[k]*(XP).reduce(II[k])]
    return ideal(R,P).groebner_basis()



#Input:   f∈ Q[x], t ∈ N
#Output:  One binomial in ⟨f⟩_{≤t} or 0 if there is none
def BinUnivariate(f,t):
    R.<x>=QQ[]
    for i in range(1,t+1):
        if R(x^i).mod(f) in QQ:
            return R(x^i)-R(x^i).mod(f)
    return 0



#Input:   f∈ Q[x], t ∈ N
#Output:  One trinomial in ⟨f⟩_{≤t} or 0 if there is none
def Trinomial(f,t):
    R.<x>=QQ[]
    f=R(f)
    if(not BinUnivariate(f,t)==0):
         return BinUnivariate(f,t)^2
    n=f.degree()
    S=dict()
    g=R(1)
    for i in range(1,t):
        g=(g*x).mod(f)
        if g in QQ:
            return BinUnivariate(f)
        h=(g-g.constant_coefficient())/g.lc()
        if h not in S:
            S[h]=[i,g.lc()]
        else:
            s= x^i-(g.lc()/S[h][1])*x^S[h][0]
            ss=s-s.mod(f)
            return ss 
    return 0



#Input:   Ideal I,  L a basis of a lattice ⊆ Z^k
#Output:  Ideal J = I ∩ Q[L]
def getJ(I,L):
    I=I.groebner_basis()
    R=I.ring()
    A=R.gens()
    n=R.ngens()
    d=len(L)
    char=['a','b','c','d','e','f','g'][:d]
    LR=LaurentPolynomialRing(QQ,list(A)+char,order='lex') 
    Rbig=PolynomialRing(QQ,list(A)+char,order='lex') 
    Rnew=PolynomialRing(QQ,char,order='lex') 
    MakeJ=list(I.groebner_basis())
    for v in range(d):
        mon=LR(1)
        for e in range(n):
            mon*=( LR(A[e])^(L[v][e]) )
        MakeJ+=[LR(char[v])- mon]
    GB=ideal(LR,MakeJ).groebner_basis()
    genJ=[]
    for g in GB:
        if Rbig(g).lm()< LR(A[n-1]):
            genJ+=[Rnew(g)]
    return ideal(Rnew,genJ).groebner_basis()

In [2]:
#Helpful: J,C=A.jordan_form(subdivide=False,transformation=True)
#Helpful: K.<q,i>=NumberField([x^2-5,x^2+1])
#Helpful: I.groebner_fan().reduced_groebner_bases()